In [2]:
%%capture 
!pip install wikidataintegrator
from rdflib import Graph, URIRef, Literal, Namespace, BNode
from rdflib.namespace import RDF, RDFS, SKOS, XSD, OWL, PROV
from wikidataintegrator import wdi_core, wdi_config
import uuid
import urllib.parse

In [3]:
wdi_config.property_value_types

{'commonsMedia': 'http://wikiba.se/ontology#CommonsMedia',
 'external-id': 'http://wikiba.se/ontology#ExternalId',
 'geo-shape': 'http://wikiba.se/ontology#GeoShape',
 'globe-coordinate': 'http://wikiba.se/ontology#GlobeCoordinate',
 'math': 'http://wikiba.se/ontology#Math',
 'monolingualtext': 'http://wikiba.se/ontology#Monolingualtext',
 'quantity': 'http://wikiba.se/ontology#Quantity',
 'string': 'http://wikiba.se/ontology#String',
 'tabular-data': 'http://wikiba.se/ontology#TabularData',
 'time': 'http://wikiba.se/ontology#Time',
 'edtf': '<http://wikiba.se/ontology#Edtf>',
 'url': 'http://wikiba.se/ontology#Url',
 'wikibase-item': 'http://wikiba.se/ontology#WikibaseItem',
 'wikibase-property': 'http://wikiba.se/ontology#WikibaseProperty',
 'lexeme': 'http://wikiba.se/ontology#WikibaseLexeme',
 'form': 'http://wikiba.se/ontology#WikibaseForm',
 'sense': 'http://wikiba.se/ontology#WikibaseSense',
 'musical-notation': 'http://wikiba.se/ontology#MusicalNotation'}

In [4]:
rdf_item = Graph()
ns = dict()
for prefix in wdi_config.prefix.keys():
    ns[prefix] = Namespace( wdi_config.prefix[prefix])
    rdf_item.namespace_manager.bind(prefix, ns[prefix])

In [21]:
#qid = "Q35869"
qid = "Q38"
item = wdi_core.WDItemEngine(wd_item_id=qid)
json_item = item.get_wd_json_representation()
rdf_item.add((ns["wd"][qid], RDF.type, ns["wikibase"].Item)) 

In [20]:
import requests
import json

json_item2 = json.loads(requests.get("http://www.wikidata.org/entity/"+qid+".json").text)["entities"][qid]


TypeError: unsupported operand type(s) for -: 'dict' and 'dict'

In [22]:
def makeWdt(claim, preferredSet, value):
    if preferredSet:
                    if claim["rank"] == "preferred":
                        rdf_item.add((ns["wd"][qid], ns["wdt"][pid], value))
    else:
                    if claim["rank"] == "normal":
                        rdf_item.add((ns["wd"][qid], ns["wdt"][pid], value))

In [23]:
properties = dict()
linked_items = []

def owlPropertyTypes(owlType):
    rdf_item.add((ns["wd"][pid], RDF.type, owlType))
    rdf_item.add((ns["p"][pid], RDF.type, owlType))
    rdf_item.add((ns["wdtn"][pid], RDF.type, owlType))
    rdf_item.add((ns["wdt"][pid], RDF.type, owlType))
    rdf_item.add((ns["pq"][pid], RDF.type, owlType))
    rdf_item.add((ns["pqn"][pid], RDF.type, owlType))
    rdf_item.add((ns["pqv"][pid], RDF.type, owlType))
    rdf_item.add((ns["pr"][pid], RDF.type, owlType))
    rdf_item.add((ns["prn"][pid], RDF.type, owlType))
    rdf_item.add((ns["prv"][pid], RDF.type,owlType))
    rdf_item.add((ns["ps"][pid], RDF.type, owlType))
    rdf_item.add((ns["psn"][pid], RDF.type, owlType))
    rdf_item.add((ns["psv"][pid], RDF.type, owlType))

def propdefs(pid, item, datatype):
    ## Properties and their derivatives
    object_properties = ["wikibase-item", 'external-id', 'string', 'commonsMedia',  'time', 'edtf', 'globe-coordinate', 'url', 'quantity', 'wikibase-property', 'monolingualtext', 'math', 'tabular-data', 'form', 'lexeme', 'geo-shape', 'musical-notation', 'sense']
    data_properties = ['external-id', 'string', 'time', 'edtf', 'globe-coordinate', 'quantity', 'monolingualtext', 'math', 'geo-shape', 'form', 'lexeme', 'musical-notation', 'sense' ]
    print(pid)  
    # ObjectProperty
    if datatype in object_properties:
        owlPropertyTypes(OWL.ObjectProperty)
    # Data Properties
    if datatype in data_properties:
        owlPropertyTypes(OWL.DatatypeProperty)  
    rdf_item.add((ns["wd"][pid], RDF.type, ns["wikibase"].Property))
    rdf_item.add((ns["wd"][pid], ns["wikibase"].directClaim, ns["wdt"][pid]))
    rdf_item.add((ns["wd"][pid], ns["wikibase"].claim, ns["p"][pid]))
    rdf_item.add((ns["wd"][pid], ns["wikibase"].statementProperty, ns["ps"][pid]))
    rdf_item.add((ns["wd"][pid], ns["wikibase"].statementValue, ns["psv"][pid]))
    rdf_item.add((ns["wd"][pid], ns["wikibase"].qualifier, ns["pq"][pid]))
    rdf_item.add((ns["wd"][pid], ns["wikibase"].qualifierValue, ns["pqv"][pid]))
    rdf_item.add((ns["wd"][pid], ns["wikibase"].reference, ns["pr"][pid]))
    rdf_item.add((ns["wd"][pid], ns["wikibase"].referenceValue, ns["prv"][pid]))
    rdf_item.add((ns["wd"][pid], ns["wikibase"].novalue, ns["wdno"][pid]))
    rdf_item.add((ns["wd"][pid], ns["wikibase"].directClaimNormalized, ns["wdtn"][pid]))
    rdf_item.add((ns["wd"][pid], ns["wikibase"].qualifierValueNormalized, ns["pqn"][pid]))
    rdf_item.add((ns["wd"][pid], ns["wikibase"].referenceValueNormalized, ns["prn"][pid]))
    rdf_item.add((ns["wd"][pid], ns["wikibase"].statementValueNormalized, ns["psn"][pid]))
    rdf_item.add((ns["wd"][pid], ns["wikibase"].propertyType, URIRef(wdi_config.property_value_types[datatype])))
    rdf_item.add((ns["wdno"][pid], RDF.type, OWL.Class))
    owl_restriction = BNode()
    rdf_item.add((owl_restriction, RDF.type, OWL.Restriction))
    rdf_item.add((owl_restriction, OWL.onProperty, ns["wdt"][pid]))
    rdf_item.add((owl_restriction, OWL.someValuesFrom, OWL.Thing))    
    rdf_item.add((ns["wdno"][pid], OWL.complementOf, owl_restriction)) 
    
def parseSnak(statement):
    value = statement["datavalue"]["value"]
    if statement["datatype"] == "commonsMedia":
        return URIRef("http://commons.wikimedia.org/wiki/Special:FilePath/"+value.replace(" ", "_"))
    elif statement["datatype"] == "string": return Literal(value)   
    elif statement["datatype"] == "external-id": return Literal(value)
    elif statement["datatype"] == "wikibase-item": return ns["wd"][value]
    elif statement["datatype"] == "monolingualtext": return Literal(value["text"], value["language"])
    elif statement["datatype"] == "geo-shape": return URIRef("http://commons.wikimedia.org/data/main/"+value)
    elif statement["datatype"] == "globe-coordinate":
        latitude = value["latitude"]
        longitude = value["longitude"]
        # altitude = claim["mainsnak"]["datavalue"]["value"]["altitude"] # not used
        precision = value["precision"] # not used
        globe = value["globe"]   # not used
        return Literal("Point("+str(longitude)+","+str(latitude)+")", datatype=ns["geo"].wktLiteral)
    elif statement["datatype"] == "quantity":
        amount = value["amount"]
        unit =  value["unit"] 
        return Literal(value["amount"], datatype=XSD.decimal)
    elif statement["datatype"] == "time": return Literal(value["time"], datatype=XSD.dateTime)
    elif statement["datatype"] == "url": return URIRef(value) 
    else: raise ValueError('unknown snak datatype ' + statement["datatype"])

for pid in json_item['claims'].keys():
    if pid not in properties.keys():
        properties[pid] = json_item['claims'][pid][0]["mainsnak"]["datatype"]
    ## Ststements 
    for claim in json_item['claims'][pid]:  
        statement_uri = ns["s"][claim["id"]]
        #rank
        if claim["rank"] == "normal": 
            rdf_item.add((statement_uri, ns["wikibase"].rank, ns["wikibase"].NormalRank))
        if claim["rank"] == "preferred":
            rdf_item.add((statement_uri, ns["wikibase"].rank, ns["wikibase"].PreferredRank))
        if claim["rank"] == "deprecated":
            rdf_item.add((statement_uri, ns["wikibase"].rank, ns["wikibase"].DeprecatedRank))
        
        # values   
        for claim2 in json_item['claims'][pid]:
            if claim2["rank"] == "preferred":
                preferredSet = True
                break
        else:
            preferredSet = False
        
        ## first no value
        if claim["mainsnak"]["snaktype"] == "novalue":
            rdf_item.add((statement_uri, RDF.type, ns["wdno"][pid]))
        else: 
            object = parseSnak(claim["mainsnak"])
            rdf_item.add((statement_uri, ns["ps"][pid], object))
            makeWdt(claim, preferredSet, object)
        
        rdf_item.add((ns["wd"][qid],ns["p"][pid], statement_uri))
        rdf_item.add((statement_uri,RDF.type, ns["wikibase"].Statement))
        
        if preferredSet:
            if claim["rank"] == "preferred":
                  rdf_item.add((statement_uri,RDF.type, ns["wikibase"].BestRank))
        else:
            if claim["rank"] == "normal":
                  rdf_item.add((statement_uri,RDF.type, ns["wikibase"].BestRank))
                    
        
        #qualifiers
        for qualifier in claim["qualifiers"].keys():
            if qualifier not in properties.keys():
                properties[qualifier] = claim["qualifiers"][qualifier][0]["datatype"]
            for qualifier_prop in claim["qualifiers"][qualifier]:
                object = parseSnak(qualifier_prop)
                rdf_item.add((statement_uri, ns["pq"][pid], object))
                
        #references
        for reference in claim["references"]:
            reference_uri = ns["ref"][reference["hash"]]
            rdf_item.add((reference_uri, RDF.type, ns["wikibase"].Reference))
            rdf_item.add((statement_uri, PROV.wasDerivedFrom, reference_uri))

            for ref_prop in reference["snaks"].keys():
                if ref_prop not in properties.keys():
                    properties[ref_prop] = reference["snaks"][ref_prop][0]["datatype"]
                for ref_prop_statement in reference["snaks"][ref_prop]:
                    object = parseSnak(ref_prop_statement)
                    rdf_item.add((statement_uri, ns["pr"][pid], object))

In [24]:
# sitelinks
for sitelink in json_item['sitelinks'].keys():
    wiki = URIRef(json_item['sitelinks'][sitelink]["url"])
    print(json_item['sitelinks'][sitelink]["url"])
    partof = URIRef(json_item['sitelinks'][sitelink]["url"].split("wiki")[0])
    language = sitelink.replace("wiki", "")
    rdf_item.add((wiki, RDF.type, ns["schema"].Article))
    rdf_item.add((wiki, schema.about, ns["wd"][qid]))
    rdf_item.add((wiki, schema.inLanguage, ns["wd"][qid]))
    rdf_item.add((wiki, schema.inPartOf, partof))

In [9]:
json_item['sitelinks']

{}

In [25]:
def transformLabels(qid, json_item):
    # Heading
    for language in json_item["labels"].keys():
        rdf_item.add((ns["wd"][qid], RDFS.label, Literal(json_item["labels"][language]["value"], language)))
        rdf_item.add((ns["wd"][qid], ns["schema"].name, Literal(json_item["labels"][language]["value"], language)))
        rdf_item.add((ns["wd"][qid], ns["skos"].prefLabel, Literal(json_item["labels"][language]["value"], language)))

    for language in json_item["descriptions"].keys():
        rdf_item.add((ns["wd"][qid], ns["schema"].description, Literal(json_item["descriptions"][language]["value"], language)))

    for language in json_item["aliases"].keys():
        for label in json_item["aliases"][language]:
            rdf_item.add((ns["wd"][qid], SKOS.altLabel, Literal(label, language))) 
                          
transformLabels(qid, json_item)     
    
for pid in properties.keys():
    pid_item = wdi_core.WDItemEngine(wd_item_id=pid).get_wd_json_representation()
    transformLabels(pid, pid_item)
    propdefs(pid, pid_item, properties[pid])



P3529
P2924
P1082
P585
P248
P854
P813
P1667
P1151
P1333
P5125
P349
P38
P580
P143
P582
P3479
P3221
P2581
P1566
P227
P487
P1792
P2852
P366
P1680
P98
P1476
P3916
P2853
P395
P3097
P3238
P1335
P2633
P298
P1313
P213
P3417
P2436
P2144
P417
P17
P3219
P486
P2959
P3452
P3106
P122
P982
P832
P1552
P3680
P94
P2981
P2573
P30
P518
P3422
P5698
P530
P805
P531
P571
P577
P304
P957
P1906
P935
P194
P299
P605
P2184
P4613
P3911
P4390
P1365
P1705
P3984
P407
P1810
P206
P36
P856
P237
P3271
P1549
P2997
P949
P402
P5008
P3068
P1332
P1334
P691
P5905
P2258
P4841
P459
P2979
P1225
P208
P3270
P3000
P1013
P910
P3612
P35
P123
P1065
P155
P1791
P242
P195
P2061
P1617
P3896
P1036
P1081
P2046
P3743
P6
P984
P463
P2163
P4342
P1198
P214
P5658
P4656
P4427
P5573
P1417
P2950
P706
P948
P1740
P2884
P1448
P421
P1264
P1465
P2347
P85
P51
P209
P2250
P361
P373
P78
P47
P654
P6150
P31
P610
P2044
P1464
P4672
P1813
P1622
P2982
P1589
P625
P5019
P898
P5180
P37
P443
P5247
P150
P92
P3024
P474
P646
P3175
P5198
P244
P998
P163
P901
P3509
P1343
P41
P

In [26]:
for linked_qid in linked_items:
    print(linked_qid)
    rdf_item.add((ns["wd"][linked_qid], RDF.type, ns["wikibase"].Item))
    linked_qid_item = wdi_core.WDItemEngine(wd_item_id=linked_qid).get_wd_json_representation()
    transformLabels(linked_qid, linked_qid_item)

In [27]:
compareRDF = Graph()
compareRDF.parse("http://www.wikidata.org/entity/"+qid+".ttl")

<Graph identifier=N29812598d51b4259a61b0faa50791d5b (<class 'rdflib.graph.Graph'>)>

In [28]:
diffRdf = compareRDF-rdf_item
for prefix in wdi_config.prefix.keys():
    ns[prefix] = Namespace( wdi_config.prefix[prefix])
    diffRdf.namespace_manager.bind(prefix, ns[prefix])
diffRdf.serialize(format="turtle", destination="1.ttl")

In [ ]:
rdf_item.serialize(format="turtle", destination="2.ttl")

In [ ]:
rdf_item.serialize(format="turtle", destination="3.ttl")

In [ ]:
json_item